In [1]:
import os
import re

import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import treebank
from nltk.corpus.reader.conll import *

from pymystem3 import Mystem

import pymorphy2


Dataset - a lecture tapescript on socioligical studies

In [2]:
with open('2005_04_14_miller_.txt') as fl:
    text = fl.read()

Text preprocessing

In [5]:
tokenized_text = TreebankWordTokenizer().tokenize(text.lower())
wordsFiltered = []

for w in tokenized_text:
    for wf in re.findall(r'(?u)(\w+)', w):
        for wfreal in re.findall(r'(\D+)', wf):
            wordsFiltered.append(''.join(wfreal).replace('\n',''))
            

In [6]:
unique_words = list(set(wordsFiltered))
print(unique_words[:50])

['процентному', 'империи', 'после', 'нет', 'течение', 'никита', 'одно', 'правит', 'польскими', 'немецким', 'значения', 'увидим', 'упорно', 'скажу', 'пожалуй', 'книг', 'француз', 'стремились', 'безграмотности', 'прошло', 'молодые', 'империю', 'сепаратистски', 'велика', 'опубликовать', 'пересказывать', 'пользовался', 'выгона', 'носил', 'думает', 'почитайте', 'присваивали', 'соперников', 'мадьяризации', 'прошлого', 'разговора', 'пусть', 'зала', 'равно', 'мыслю', 'фактору', 'историку', 'обошлись', 'фантомизировать', 'ничего', 'этому', 'неперевариваемой', 'взамен', 'угро', 'условиях']


## POS_taggers comparison

### Mystem from Pymystem3

- input format: string _(can prosess the whole text)_
- output format: dictionary with pos tags + additional grammatical info and lemma
- method: mystem.analyze(text)

In [7]:
m = Mystem()

In [8]:
#example
a = m.analyze('стали')
a

[{'analysis': [{'gr': 'V,нп=прош,мн,изъяв,сов', 'lex': 'становиться'}],
  'text': 'стали'},
 {'text': '\n'}]

In [9]:
POS_mystem = m.analyze(text)

In [10]:
POS_mystem[:20]

[{'analysis': [{'gr': 'SPRO,мн,1-л=им', 'lex': 'мы'}], 'text': 'Мы'},
 {'text': ' ('},
 {'analysis': [{'gr': 'PR=', 'lex': 'с'}], 'text': 'с'},
 {'text': ' '},
 {'analysis': [{'gr': 'APRO=(пр,ед,жен|дат,ед,жен|род,ед,жен|твор,ед,жен)',
    'lex': 'некоторый'}],
  'text': 'некоторой'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,жен,неод=твор,ед', 'lex': 'задержка'}],
  'text': 'задержкой'},
 {'text': ', '},
 {'analysis': [{'gr': 'V,пе=(прош,пр,ед,прич,полн,жен,сов,страд|прош,дат,ед,прич,полн,жен,сов,страд|прош,род,ед,прич,полн,жен,сов,страд|прош,твор,ед,прич,полн,жен,сов,страд)',
    'lex': 'вызывать'}],
  'text': 'вызванной'},
 {'text': ' '},
 {'analysis': [{'gr': 'A=твор,мн,полн', 'lex': 'технический'}],
  'text': 'техническими'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,сред,неод=твор,мн', 'lex': 'обстоятельство'}],
  'text': 'обстоятельствами'},
 {'text': ') '},
 {'analysis': [{'gr': 'V,несов,пе=(непрош,мн,изъяв,1-л|непрош,ед,прич,кр,муж,страд)',
    'lex': 'публиковать'}],
  'text': 

In [11]:
POS_mystem2 = []

for word in unique_words:
    POS_mystem2.append(m.analyze(word))

In [12]:
POS_mystem2[:10]

[[{'analysis': [{'gr': 'A=(дат,ед,полн,муж|дат,ед,полн,сред)',
     'lex': 'процентный'}],
   'text': 'процентному'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)',
     'lex': 'империя'}],
   'text': 'империи'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'ADV=', 'lex': 'после'}], 'text': 'после'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'PART=', 'lex': 'нет'}], 'text': 'нет'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'S,сред,неод=(вин,ед|им,ед)', 'lex': 'течение'}],
   'text': 'течение'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'S,имя,муж,од=им,ед', 'lex': 'никита'}],
   'text': 'никита'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'APRO=(вин,ед,сред|им,ед,сред)', 'lex': 'один'}],
   'text': 'одно'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'V,несов,пе=непрош,ед,изъяв,3-л', 'lex': 'править'}],
   'text': 'правит'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'A=твор,мн,полн', 'lex': 'польский'}],
   'text': 'польскими'},
  {'text': '\n'}],

As we can see in the results, morpological ambiguity is most common for nouns in objective cases as well as adjectives and participals. The results for the whole text parsing are the same as for the parsing of the separate words which means that Mystem doesn't take context into account to resolve ambiguity. Plus when it comes to interPOS ambiguity Mystems seems to show only one option with the highest probability (see example with "стали").

### Pymorhpy2

- input format: string _(one word only)_
- output format: list of pymorphy2.analyzer objects with pos tags + additional grammatical info and lemma
- method: pymorphy2.MorphAnalyzer().parse('str')

In [13]:
morph = pymorphy2.MorphAnalyzer()

In [14]:
#example
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [15]:
POS_PM2 = []

for word in tokenized_text:
    POS_PM2.append(morph.parse(word))

In [16]:
POS_PM2[:5]

[[Parse(word='мы', tag=OpencorporaTag('NPRO,1per plur,nomn'), normal_form='мы', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'мы', 1990, 0),))],
 [Parse(word='(', tag=OpencorporaTag('PNCT'), normal_form='(', score=1.0, methods_stack=((<PunctuationAnalyzer>, '('),))],
 [Parse(word='с', tag=OpencorporaTag('PREP'), normal_form='с', score=0.998363, methods_stack=((<DictionaryAnalyzer>, 'с', 375, 0),)),
  Parse(word='с', tag=OpencorporaTag('NOUN,inan,femn,Fixd,Abbr sing,nomn'), normal_form='секунда', score=6.8e-05, methods_stack=((<DictionaryAnalyzer>, 'с', 2747, 13),)),
  Parse(word='с', tag=OpencorporaTag('NOUN,inan,femn,Fixd,Abbr sing,gent'), normal_form='секунда', score=6.8e-05, methods_stack=((<DictionaryAnalyzer>, 'с', 2747, 14),)),
  Parse(word='с', tag=OpencorporaTag('NOUN,inan,femn,Fixd,Abbr sing,datv'), normal_form='секунда', score=6.8e-05, methods_stack=((<DictionaryAnalyzer>, 'с', 2747, 15),)),
  Parse(word='с', tag=OpencorporaTag('NOUN,inan,femn,Fixd,Abbr sing,accs'), norma

Pymorhpy2 shows great results in showing of all the possible interpretations of the wordform but again since it is dictionary-based it has no tool for morphological disambiguation.

### nltk.tag

- input format: list of strings
- output format: list of tuples ('wordform', 'POStag') 
- method: tnt.TnT().tag() after training on a tagged corpus

In [17]:
train_file = 'unamb_sent_14_6.conllu'
train_reader = ConllCorpusReader(root = '', fileids = [train_file], columntypes = ['ignore', 'words', 'ignore', 'pos', 'chunk'])


sents = list(train_reader.iob_sents())
train_sents = sents[:30000]
test_sents = sents[30000:]

print(len(train_sents))
print(len(test_sents))

30000
8508


In [18]:
train_sents[1]

[('Сохранится', 'VERB', '_'),
 ('ли', 'PART', '_'),
 ('градус', 'NOUN', '_'),
 ('дискуссии', 'NOUN', '_'),
 ('в', 'ADP', '_'),
 ('новом', 'ADJ', '_'),
 ('сезоне', 'NOUN', '_'),
 ('?', 'PUNCT', '_')]

In [19]:
from nltk.tag import tnt

test_sents_1 = []
for sent in test_sents:
    sent_1 = []
    for word in sent:
        sent_1.append(word[:2])
    test_sents_1.append(sent_1)
print(test_sents_1[0])

train_sents_1 = []
for sent in test_sents:
    sent_1 = []
    for word in sent:
        sent_1.append(word[:2])
    train_sents_1.append(sent_1)
print(test_sents_1[0])

[('Письмо', 'NOUN'), ('восьмое', 'ADJ'), ('посвящено', 'ADJ'), ('правилам', 'NOUN'), (',', 'PUNCT'), ('благоприятным', 'ADJ'), ('для', 'ADP'), ('всевозможных', 'ADJ'), ('грешников', 'NOUN'), (':', 'PUNCT'), ('воров', 'NOUN'), (',', 'PUNCT'), ('банкротов', 'NOUN'), (',', 'PUNCT'), ('нечестных', 'ADJ'), ('судей', 'NOUN'), (',', 'PUNCT'), ('распутных', 'ADJ'), ('женщин', 'NOUN'), ('.', 'PUNCT')]
[('Письмо', 'NOUN'), ('восьмое', 'ADJ'), ('посвящено', 'ADJ'), ('правилам', 'NOUN'), (',', 'PUNCT'), ('благоприятным', 'ADJ'), ('для', 'ADP'), ('всевозможных', 'ADJ'), ('грешников', 'NOUN'), (':', 'PUNCT'), ('воров', 'NOUN'), (',', 'PUNCT'), ('банкротов', 'NOUN'), (',', 'PUNCT'), ('нечестных', 'ADJ'), ('судей', 'NOUN'), (',', 'PUNCT'), ('распутных', 'ADJ'), ('женщин', 'NOUN'), ('.', 'PUNCT')]


In [21]:
tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_sents_1)
tnt_pos_tagger.evaluate(test_sents_1)

0.9953090882497362

In [24]:
%time
POS_NLTK = tnt_pos_tagger.tag(tokenized_text[:500])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.4 µs


In [26]:
print(POS_NLTK)

[('мы', 'PRON'), ('(', 'PUNCT'), ('с', 'ADP'), ('некоторой', 'DET'), ('задержкой', 'Unk'), (',', 'PUNCT'), ('вызванной', 'Unk'), ('техническими', 'ADJ'), ('обстоятельствами', 'Unk'), (')', 'PUNCT'), ('публикуем', 'Unk'), ('полную', 'ADJ'), ('стенограмму', 'Unk'), ('лекции', 'Unk'), ('историка', 'Unk'), ('алексея', 'Unk'), ('миллера', 'Unk'), (',', 'PUNCT'), ('прочитанной', 'Unk'), ('31', 'NUM'), ('марта', 'NOUN'), ('2005', 'NUM'), ('года', 'NOUN'), ('в', 'ADP'), ('клубе-литературном', 'Unk'), ('кафе', 'NOUN'), ('bilingua', 'Unk'), ('в', 'ADP'), ('рамках', 'NOUN'), ('проекта', 'NOUN'), ('«', 'PUNCT'), ('публичные', 'Unk'), ('лекции', 'Unk'), ('полит.ру', 'Unk'), ('»', 'PUNCT'), ('.лекция', 'Unk'), ('алексея', 'Unk'), ('миллера', 'Unk'), ('–', 'PUNCT'), ('это', 'PART'), ('результат', 'NOUN'), ('ясного', 'Unk'), ('и', 'CONJ'), ('научно', 'Unk'), ('отстраненного', 'Unk'), ('анализа', 'NOUN'), ('тем', 'Unk'), (',', 'PUNCT'), ('которые', 'DET'), ('почти', 'Unk'), ('никогда', 'ADV'), ('в', 'A

In [39]:
unk = []

for tag in POS_NLTK:
    if tag[1] == 'Unk':
        unk.append(tag)
    else:
        continue

In [41]:
print(len(unk), '\n', unk)

133 
 [('задержкой', 'Unk'), ('вызванной', 'Unk'), ('обстоятельствами', 'Unk'), ('публикуем', 'Unk'), ('стенограмму', 'Unk'), ('лекции', 'Unk'), ('историка', 'Unk'), ('алексея', 'Unk'), ('миллера', 'Unk'), ('прочитанной', 'Unk'), ('клубе-литературном', 'Unk'), ('bilingua', 'Unk'), ('публичные', 'Unk'), ('лекции', 'Unk'), ('полит.ру', 'Unk'), ('.лекция', 'Unk'), ('алексея', 'Unk'), ('миллера', 'Unk'), ('ясного', 'Unk'), ('научно', 'Unk'), ('отстраненного', 'Unk'), ('тем', 'Unk'), ('почти', 'Unk'), ('острого', 'Unk'), ('желания', 'Unk'), ('набить', 'Unk'), ('морды', 'Unk'), ('обсуждении.', 'Unk'), ('идеологиях', 'Unk'), ('империи.', 'Unk'), ('идейных', 'Unk'), ('авторство', 'Unk'), ('данной', 'Unk'), ('поименно', 'Unk'), ('удачи', 'Unk'), ('неудачи', 'Unk'), ('трудности', 'Unk'), ('исполнения', 'Unk'), ('социальном', 'Unk'), ('материале.', 'Unk'), ('вообще', 'Unk'), ('историки', 'Unk'), ('обслуживают', 'Unk'), ('проекты…есть', 'Unk'), ('нация', 'Unk'), ('заблуждении', 'Unk'), ('прошлого.

In [36]:
b = tnt_pos_tagger.tag(['стали'])

In [37]:
b

[('стали', 'VERB')]

The results show precise POS-tagging but quite a few words are left untagged (133 out of 500; though most of them present named enities and junk, such frequent words as "никогда" or "могли" are also marked as unknown). So the quality of POS-tagging depends on the trainig corpus size.